In [1]:
%cd ..
%pip install -e .

/home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
Obtaining file:///home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
  Preparing metadata (setup.py) ... done
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
Note: you may need to restart the kernel to use updated packages.


In [2]:
import random

from src.models.dense import NN0
from src.models.dense import NN1
from src.models.gcn import GCN0
from src.models.gcn import GCN1

from src.models.train_model import train2
from src.models.train_model import random_splits

from src.models.evaluate_model import evaluate0

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from sklearn.manifold import TSNE

from src.models.evaluate_model import test

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=T.NormalizeFeatures())
# no normalization, using n != 0 and m!= 0 instead
# doesnt work, just use normalization instead
# dataset = Planetoid(root='data/Planetoid', name='Cora')

data = dataset[0].to(device)
data = random_splits(data, 50, 20)
splits = data.train_mask, data.val_mask, data.test_mask

print(f'train size: {data.train_mask.sum()}')
print(f'valid size: {data.valid_mask.sum()}')
print(f'test size: {data.test_mask.sum()}')

for mu in range(10):
    mu = mu / 10
    
    torch.manual_seed(1)
    random.seed(1)

    gcn_model = GCN1(num_node_features=dataset.num_node_features,
                     num_classes=dataset.num_classes) \
                    .to(device)

    gcn_model = train2(gcn_model, data, mu=mu, num_epochs=350)
    # gcn_model = train2(gcn_model, data)
    acc = evaluate0(gcn_model, data)
    # print(f'mu: {mu}, reg, Accuracy: {acc:.4f}')
    train_acc, val_acc, test_acc = test(gcn_model, data, splits)
    print(f'mu: {mu}, train_acc: {train_acc}, val_acc: {val_acc}, test_acc: {test_acc}')
    print('--------------------------------------------')

train size: 350
valid size: 140
test size: 2218
mu: 0.0, train_acc: 0.14285714285714285, val_acc: 0.156, test_acc: 0.15689810640216412
--------------------------------------------
mu: 0.1, train_acc: 0.14285714285714285, val_acc: 0.072, test_acc: 0.06627592425608657
--------------------------------------------


KeyboardInterrupt: 